### The modules we will need

In [1]:
# We import the SQLite python module for the database handling
import sqlite3

# the bz2 module is used to uncompress the files
import bz2

# the csv module serves for manipulating csv files
import csv

# while the OS module will help us manage the filepaths
import os

# pandas will be useful for storing and processing query results
import pandas as pd


### We uncompress the bz2 file 

In [2]:
# The current working directory
cwd = os.getcwd()

# compressed file name
compressed = '2008.csv.bz2'

# The filepath of the compressed file
filepath_to_compressed = cwd + '/' + 'input' + '/' + compressed

In [3]:
# We first check whether the csv file already exists, if so we pass
# If not, we open the file, uncompress it and then write a new csv file.

if os.path.isfile(filepath_to_compressed[:-4]):    # isfile returns True if the filepath_to_compressed[:-4] leads to a file
    print ('File already exists')

else:
    zipfile = bz2.BZ2File(filepath_to_compressed)    # open the file
    data = zipfile.read()    # get the decompressed data. Pay attention, it is in binary format
    newfilepath = filepath[:-4]    # cut away the .bz2 ending
    open(newfilepath, 'wb').write(data)    # tranform the uncompressed binary data into a csv file.
    
    print('File' + filepath + 'created')


File already exists


### Now that we have all the relevant data we can start building our database

In [4]:
# We first create the database and the tables inside it


## The relevant filepaths
database_name = 'air_flights_project.db'
filepath_to_database = cwd + '/' + 'output' + '/' + database_name

creation_script_name = 'database_creation_script'
filepath_to_creation_script = cwd + '/' + 'output' + '/' + creation_script_name


## We transform our file into a long string, to be fed to sqlite
creation_string = ''
with open(filepath_to_creation_script,'r') as creation_script:
    for line in creation_script:
        creation_string += line


## creation of the database 
conn = sqlite3.connect(filepath_to_database)  # connect if exists, otherwise create database
cursor = conn.cursor()
cursor.executescript(creation_string)  # The script will create tables only if they do not exist -IF NOT EXIST clause

conn.close()

### Now we can insert the data into the various tables

In [5]:
# TABLE 1
# CARRIERS


# 1 - The relevant filepaths

# recall that filepath_to_database = cwd + '/' + 'output' + '/' + database_name
filepath_to_carriers = cwd + '/' + 'input' + '/' + 'carriers.csv'



# 2 - open the connection and clear the contents of the table
conn = sqlite3.connect(filepath_to_database)
cursor = conn.cursor()
query = ''' delete from carriers  '''
cursor.execute(query)

# 3 - read the lines 
with open(filepath_to_carriers,'r') as carriers:
    carriers_reader = csv.reader(carriers,delimiter=",")
    next(carriers_reader)  # skip first line with column names using the reader's next method
    
    for line in carriers_reader:

        query = ''' insert into carriers (Code, Full_name ) values (?, ?) '''
        cursor.execute(query, [line[0], line[1]])



conn.commit()



query =  ''' select * from carriers'''

carriers_df = pd.read_sql_query(query, conn, index_col=None, coerce_float=False, parse_dates=None, chunksize=None)

conn.close()

In [6]:
carriers_df

,Code,Full_name
0,02Q,Titan Airways
1,04Q,Tradewind Aviation
2,05Q,"Comlux Aviation, AG"
3,06Q,Master Top Linhas Aereas Ltd.
4,07Q,Flair Airlines Ltd.
5,09Q,"Swift Air, LLC"
6,0BQ,DCA
7,0CQ,ACM AIR CHARTER GmbH
8,0FQ,"Maine Aviation Aircraft Charter, LLC"
9,0GQ,"Inter Island Airways, d/b/a Inter Island Air"


### A simpe test to verify that all the data  has been loaded

In [7]:
def count_csv_rows(filepath):
    '''
    a simple way to count the lines in a csv
    '''
    with open(filepath) as csv_file:
        return ( sum(1 for line in csv_file) )

In [8]:
def count_database_rows(filepath, table_name):
    '''
    a simple way to count the rows in a table
    '''
    conn = sqlite3.connect(filepath)
    cursor = conn.cursor()
    query = '''select count(*) from ''' + table_name
    cursor.execute(query)
    return ( cursor.fetchall()[0][0] )

In [9]:
# the lines in the source csv should be one greater than the rows in the database table
count_csv_rows ( filepath_to_carriers ) == 1 + count_database_rows ( filepath_to_database, 'carriers' )


True

In [10]:
# TABLE 2
# PLANES


# 1 - The relevant filepaths
# recall that filepath_to_database = cwd + '/' + 'output' + '/' + database_name
filepath_to_planes = cwd + '/' + 'input' + '/' + 'plane-data.csv'



# 2 - open the connection and clear the contents of the table
conn = sqlite3.connect(filepath_to_database)
cursor = conn.cursor()
query = ''' delete from planes  '''
cursor.execute(query)



# 3 - read the lines and insert them into the database
with open(filepath_to_planes,'r') as planes:
    planes_reader = csv.reader(planes,delimiter=",")
    next(planes_reader)  # skip first line with column names using the reader's next method
    
    for line in planes_reader:
        
        
        if len(line) == 9:  # checking if all fields are present in the csv. There are many rows with only a TailNum
            
           
        
            # The last column in the csv contains four different kinds of values
            # 'None', valid date strings - '1969', technically valid but inappropriate date string - '0000', and empty fields
            # we try to capture various cases and transform them accordingly
            type_none = (lambda x: None if ( (x == 'None') or  (x == '0000') or (x == 0)) else int(x)) 

            # For Issue-date we need a little manipulation to get to this date format YYYY-MM-DD, which sqlite understands
            date_manipulation = (lambda x: None if (x == 'None') else ( x[6:10] + '-' + x[0:2] + '-' + x[3:5]) )
            
            
            query = ''' insert into planes 
                ( TailNum, Type,  Manufacturer, Issue_date,  Model,Status,Aircraft_type ,Engine_type ,  Plane_year ) 
                values (?,?,?,?,?,?,?,?,?) '''
            
            
            
            cursor.execute(query, [line[0],
                                   line[1],
                                   line[2],
                                  date_manipulation(line[3] ),
                                   line[4],
                                   line[5],
                                   line[6],
                                   line[7],
                                   type_none(line[8]) ] )
        
        else:  # In case there is only a TailNum in the csv, we need to fill in the gaps with None types
            for element in line:
                element = element.strip('\r\n').strip('"')
                

            query = ''' insert into planes 
                ( TailNum, Type,  Manufacturer, Issue_date,  Model,Status,Aircraft_type ,Engine_type ,  Plane_year ) 
                values (?,?,?,?,?,?,?,?,?) '''
            
            cursor.execute(query, [element,None,None,None,None,None,None,None,None ] )

conn.commit()



query =  ''' select * from planes'''

planes_df = pd.read_sql_query(query, conn, index_col=None, coerce_float=False, parse_dates=None, chunksize=None)

conn.close()

In [11]:
# pandas automatically casts integers into float in the presence of NaNs
# For this reason Plane_year is integer in the database table, and becomes float in the dataframe

planes_df

,TailNum,Type,Manufacturer,Issue_date,Model,Status,Aircraft_type,Engine_type,Plane_year
0,N050AA,None,None,None,None,None,None,None,NaN
1,N051AA,None,None,None,None,None,None,None,NaN
2,N052AA,None,None,None,None,None,None,None,NaN
3,N054AA,None,None,None,None,None,None,None,NaN
4,N055AA,None,None,None,None,None,None,None,NaN
5,N056AA,None,None,None,None,None,None,None,NaN
6,N057AA,None,None,None,None,None,None,None,NaN
7,N058AA,None,None,None,None,None,None,None,NaN
8,N059AA,None,None,None,None,None,None,None,NaN
9,N060AA,None,None,None,None,None,None,None,NaN


In [12]:
# the lines in the source csv should be one greater than the rows in the database table
count_csv_rows ( filepath_to_planes ) == 1 + count_database_rows ( filepath_to_database, 'planes' )

True

In [13]:
# TABLE 3
# AIRPORTS


# 1 - The relevant filepaths
# recall that filepath_to_database = cwd + '/' + 'output' + '/' + database_name
filepath_to_airports = cwd + '/' + 'input' + '/' + 'airports.csv'


# 2 - open the connection and clear the contents of the table
conn = sqlite3.connect( filepath_to_database )
cursor = conn.cursor()
query = ''' delete from airports  '''
cursor.execute(query)


# 3 - read the lines and insert them into the database
with open(filepath_to_airports,'r') as airports:
    airports_reader = csv.reader(airports,delimiter=",")
    next(airports_reader)  # skip first line with column names using the reader's next method
    
    for line in airports_reader:
        
        if len(line) == 7:  # checking if all fields are present in the csv

            query = ''' insert into airports ( IATA,Airport_name,City,
            US_State,Country,Latitude,Longitude ) values (?,?,?,?,?,?,?) '''
            
            cursor.execute(query, [line[0],line[1],line[2],line[3],line[4],float(line[5]),float(line[6]) ] )
        else:
            print(line) # Get us informed if a line has missing fields

conn.commit()


query =  ''' select * from airports'''

airports_df = pd.read_sql_query(query, conn, index_col=None, coerce_float=False, parse_dates=None, chunksize=None)

conn.close()

In [14]:
airports_df

,IATA,Airport_name,City,US_State,Country,Latitude,Longitude
0,00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
1,00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928
2,00V,Meadow Lake,Colorado Springs,CO,USA,38.945749,-104.569893
3,01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
4,01J,Hilliard Airpark,Hilliard,FL,USA,30.688012,-81.905944
5,01M,Tishomingo County,Belmont,MS,USA,34.491667,-88.201111
6,02A,Gragg-Wade,Clanton,AL,USA,32.850487,-86.611453
7,02C,Capitol,Brookfield,WI,USA,43.087510,-88.177869
8,02G,Columbiana County,East Liverpool,OH,USA,40.673313,-80.641406
9,03D,Memphis Memorial,Memphis,MO,USA,40.447259,-92.226961


In [15]:
# the lines in the source csv should be one greater than the rows in the database table
count_csv_rows ( filepath_to_airports ) == 1 + count_database_rows ( filepath_to_database, 'airports' )

True

In [16]:
# TABLE 4
# Flights

# 1 - The relevant filepaths
# recall that filepath_to_database = cwd + '/' + 'output' + '/' + database_name
filepath_to_flights = cwd + '/' + 'input' + '/' + '2008.csv'


# 2 - open the connection and clear the contents of the table
conn = sqlite3.connect(filepath_to_database)
cursor = conn.cursor()
query = ''' delete from flights  '''
cursor.execute(query)


# 3 - read the lines and insert them into the database
with open(filepath_to_flights,'r') as flights:
    flights_reader = csv.reader(flights,delimiter=",")
    next(flights_reader)  # skip first line with column names using the reader's, next method
    
    for line in flights_reader:
  
        
        if len(line) == 29:  # checking if all fields are present in the csv


            query = ''' insert into flights (FlightYear,FlightMonth,FlightDay,DayOfWeek,DepTime,CRSDepTime,
            ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,
            CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,
            TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,
            NASDelay,SecurityDelay,LateAircraftDelay ) 
            values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?) '''
            
            my_none = (lambda x: None if ((x == 'NA') or (x == '') ) else int(x))  # We replace None-string with 0 else we cast the string into integer

            string_none = (lambda x: None if ((x == 'NA') or (x == '') or (x == 'Unknown') ) else (x))  # We replace NA or empty string with Python None
            
            #The 'railroad' here aims at putting together all the pieces of a string that SQLite can treat as timestamp
            my_timestamp = (lambda x, y, z, w: None if ((w == 'NA') or (w == '') ) else
                    (x + '-' + y.rjust(2,'0') + '-' + z.rjust(2,'0') + ' ' + w[:-2].rjust(2,'0') + ':' + w[-2:] ) ) 
            
            # we wrap the query into a try clause. Any row that might create a problem will be sent to the except part
            try:
                cursor.execute(query, [my_none(line[0]),   # FlightYear
                                       my_none(line[1]),   # FlightMonth
                                       my_none(line[2]),   # FlightDay
                                       my_none(line[3]),   # DayOfWeek
                                       my_timestamp(line[0],line[1],line[2],line[4]),   # DepTime
                                       my_timestamp(line[0],line[1],line[2],line[5]),   # CRSDepTime
                                       my_timestamp(line[0],line[1],line[2],line[6]),   # ArrTime
                                       my_timestamp(line[0],line[1],line[2],line[7]),   # CRSArrTime
                                       (line[8]),          # UniqueCarrier
                                       my_none(line[9]),   # FlightNum
                                       string_none(line[10]),         # TailNum
                                       my_none(line[11]),  # ActualElapsedTime
                                       my_none(line[12]),  # CRSElapsedTime
                                       my_none(line[13]),  # AirTime
                                       my_none(line[14]),  # ArrDelay
                                       my_none(line[15]),  # DepDelay
                                       (line[16]),         # Origin
                                       (line[17]),         # Dest
                                       my_none(line[18]),  # Distance
                                       my_none(line[19]),  # TaxiIn
                                       my_none(line[20]),  # TaxiOut
                                       (line[21]),         # Cancelled
                                       string_none(line[22]),  # CancellationCode
                                       (line[23]),         # Diverted
                                       my_none(line[24]),  # CarrierDelay
                                       my_none(line[25]),  # WeatherDelay
                                       my_none(line[26]),  # NASDelay
                                       my_none(line[27]),  # SecurityDelay
                                       my_none(line[28])]) # LateAircraftDelay
            except:  # The main error was caused by four duplicate lines that violated the uniqueness constraint
                print (line)  # Let us see the troublesome lines
     
        else:
            print(line) # Get us informed if a line has missing fields



conn.commit()



query =  ''' select * from flights limit 200'''

flights_df = pd.read_sql_query(query, conn, index_col=None, coerce_float=False, parse_dates=None, chunksize=None)

conn.close()

['2008', '2', '28', '4', '829', '835', '1341', '1350', 'F9', '728', 'N201FR', '192', '195', '172', '-9', '-6', 'DEN', 'DCA', '1476', '6', '14', '0', '', '0', 'NA', 'NA', 'NA', 'NA', 'NA']
['2008', '2', '28', '4', '1427', '1435', '1640', '1636', 'F9', '727', 'N201FR', '253', '241', '228', '4', '-8', 'DCA', 'DEN', '1476', '9', '16', '0', '', '0', 'NA', 'NA', 'NA', 'NA', 'NA']
['2008', '2', '28', '4', '1854', '1807', '1946', '1902', 'F9', '773', 'N201FR', '112', '115', '91', '44', '47', 'DEN', 'LAS', '629', '8', '13', '0', '', '0', '44', '0', '0', '0', '0']
['2008', '2', '28', '4', '2027', '1942', '2314', '2229', 'F9', '780', 'N201FR', '107', '107', '84', '45', '45', 'LAS', 'DEN', '629', '10', '13', '0', '', '0', '1', '0', '44', '0', '0']


In [17]:
flights_df

,FlightYear,FlightMonth,FlightDay,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2008-01-03 20:03,2008-01-03 19:55,2008-01-03 22:11,2008-01-03 22:25,WN,335,...,4.0,8.0,0,None,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,2008-01-03 07:54,2008-01-03 07:35,2008-01-03 10:02,2008-01-03 10:00,WN,3231,...,5.0,10.0,0,None,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,2008-01-03 06:28,2008-01-03 06:20,2008-01-03 08:04,2008-01-03 07:50,WN,448,...,3.0,17.0,0,None,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,2008-01-03 09:26,2008-01-03 09:30,2008-01-03 10:54,2008-01-03 11:00,WN,1746,...,3.0,7.0,0,None,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,2008-01-03 18:29,2008-01-03 17:55,2008-01-03 19:59,2008-01-03 19:25,WN,3920,...,3.0,10.0,0,None,0,2.0,0.0,0.0,0.0,32.0
5,2008,1,3,4,2008-01-03 19:40,2008-01-03 19:15,2008-01-03 21:21,2008-01-03 21:10,WN,378,...,4.0,10.0,0,None,0,NaN,NaN,NaN,NaN,NaN
6,2008,1,3,4,2008-01-03 19:37,2008-01-03 18:30,2008-01-03 20:37,2008-01-03 19:40,WN,509,...,3.0,7.0,0,None,0,10.0,0.0,0.0,0.0,47.0
7,2008,1,3,4,2008-01-03 10:39,2008-01-03 10:40,2008-01-03 11:32,2008-01-03 11:50,WN,535,...,7.0,7.0,0,None,0,NaN,NaN,NaN,NaN,NaN
8,2008,1,3,4,2008-01-03 06:17,2008-01-03 06:15,2008-01-03 06:52,2008-01-03 06:50,WN,11,...,6.0,19.0,0,None,0,NaN,NaN,NaN,NaN,NaN
9,2008,1,3,4,2008-01-03 16:20,2008-01-03 16:20,2008-01-03 16:39,2008-01-03 16:55,WN,810,...,3.0,6.0,0,None,0,NaN,NaN,NaN,NaN,NaN


In [18]:
# The lines in the source csv should be 5 greater than the rows in the database table.
# 1 for the header and 4 for the duplicate lines
count_csv_rows ( filepath_to_flights ) == 5 + count_database_rows ( filepath_to_database, 'flights' )


True

### After having inserted all the data, we should build custom indexes

In [19]:
# Indexes
# We will need them for the flights table, that is big and complicated.
# We have choosen the columns that are unique enough to be useful, and relevant to our possible queries

conn = sqlite3.connect(filepath_to_database)
cursor = conn.cursor()

query = '''

CREATE INDEX idx_flights_FlightDay ON flights ( FlightDay );

CREATE INDEX idx_flights_UniqueCarrier ON flights ( UniqueCarrier );

CREATE INDEX idx_flights_FlightNum ON flights ( FlightNum );

CREATE INDEX idx_flights_TailNum ON flights ( TailNum );

CREATE INDEX idx_flights_Origin ON flights ( Origin );

CREATE INDEX idx_flights_Dest ON flights ( Dest );


'''

cursor.executescript(query)

conn.commit()

conn.close()